# Notebook used to preprocessed the data for classification task

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
import re
from tqdm.notebook import tqdm

In [2]:
data_path = '../data/preprocessed_data/atp_matches_68_24_cleaned.csv'
data = pd.read_csv(data_path)
data.shape

(317049, 23)

In [3]:
#Remove all specifc Davis tourney name
data.loc[data['tourney_name'].str.contains('Davis'), 'tourney_name'] = 'Davis'

### Encode columns

In [4]:
one_hot_columns = ['tourney_name', 'surface', 'winner_ioc', 'loser_ioc', 'tourney_level', 'winner_hand', 'loser_hand']
#binary_columns = []
rankable_columns = ['round']

round_dict = {'F' : 0,
              'BR': 1,
              'SF': 2,
              'QF': 3,
              'R16': 4,
              'R32': 5,
              'R64': 6,
              'R128': 7, 
              'RR': 8, 
              'ER': 9,}

In [5]:
for c in one_hot_columns:
    data = pd.get_dummies(data, columns=[c])
data['round'] = data['round'].map(round_dict)
data.shape

(317049, 461)

### Create the target column

In [6]:
#Create columns for total games count
p = r'(\d)-(\d)'
games = data['score'].str.extractall(p).astype(float)
games.columns = ['winner', 'loser']
game_sums = games.groupby(level=0).sum()
data['winner_games'] = game_sums['winner']
data['loser_games'] = game_sums['loser']
data = data.dropna()

In [7]:
#create a game ratio score
data['game_ratio'] = data['winner_games'] / (data['loser_games'] + data['winner_games'])
#If won with less game, put a really score close to 0.5
data.loc[data['winner_games'] < data['loser_games'], 'game_ratio'] = 0.51

In [8]:
#Anonymized Winner and Loser
shuffled = np.random.rand(len(data)) < 0.5
data['player_1_age'] = np.where(shuffled, data['winner_age'], data['loser_age'])
data['player_2_age'] = np.where(shuffled, data['loser_age'], data['winner_age'])
#------
data['player_1_ht'] = np.where(shuffled, data['winner_ht'], data['loser_ht'])
data['player_2_ht'] = np.where(shuffled, data['loser_ht'], data['winner_ht'])
#------
data['player_1_seed'] = np.where(shuffled, data['winner_seed'], data['loser_seed'])
data['player_2_seed'] = np.where(shuffled, data['loser_seed'], data['winner_seed'])
#------
data['player_1_rank'] = np.where(shuffled, data['winner_rank'], data['loser_rank'])
data['player_2_rank'] = np.where(shuffled, data['loser_rank'], data['winner_rank'])
#------
data['player_1_rank_points'] = np.where(shuffled, data['winner_rank_points'], data['loser_rank_points'])
data['player_2_rank_points'] = np.where(shuffled, data['loser_rank_points'], data['winner_rank_points'])
#------
data['player_1_games'] = np.where(shuffled, data['winner_games'], data['loser_games'])
data['player_2_games'] = np.where(shuffled, data['loser_games'], data['winner_games'])

#Modify score for player 1 and 2
data.loc[(data['player_2_ht'] == data['loser_ht']) & (data['player_2_rank_points'] == data['loser_rank_points']) & (data['player_2_seed'] == data['loser_seed']) & (data['player_2_age'] == data['loser_age']), 'game_ratio'] = 1 - data.loc[(data['player_2_ht'] == data['loser_ht']) & (data['player_2_rank_points'] == data['loser_rank_points']) & (data['player_2_seed'] == data['loser_seed']) & (data['player_2_age'] == data['loser_age']), 'game_ratio']
#Remove Winner Looser columns
data = data.drop(['winner_age', 'loser_age', 'winner_ht', 'loser_ht', 'winner_seed', 'loser_seed', 'winner_rank', 'loser_rank', 'winner_rank_points', 'loser_rank_points', 'winner_games', 'loser_games'], axis=1)

# Save Dataset

In [9]:
print(data.shape)
data.to_csv('../data/preprocessed_data/atp_matches_68_24_preprocessed.csv', index=False)

(315519, 464)


In [11]:
for c in data.columns:
    print(c)

draw_size
year
month
best_of
round
score
tourney_name_ATP Rio de Janeiro
tourney_name_Acapulco
tourney_name_Adelaide
tourney_name_Adelaide 1
tourney_name_Adelaide 2
tourney_name_Almaty
tourney_name_Amersfoort
tourney_name_Amsterdam
tourney_name_Antalya
tourney_name_Antwerp
tourney_name_Astana
tourney_name_Athens
tourney_name_Athens Olympics
tourney_name_Atlanta
tourney_name_Atlanta Olympics
tourney_name_Atlanta WCT
tourney_name_Atp Cup
tourney_name_Auckland
tourney_name_Australian Open
tourney_name_Baltimore
tourney_name_Baltimore WCT
tourney_name_Bangkok
tourney_name_Banja Luka
tourney_name_Barcelona
tourney_name_Barcelona Olympics
tourney_name_Basel
tourney_name_Bastad
tourney_name_Beckenham
tourney_name_Beijing
tourney_name_Beijing Olympics
tourney_name_Belgrade
tourney_name_Belgrade 
tourney_name_Belgrade 2
tourney_name_Berlin
tourney_name_Bermuda
tourney_name_Birmingham
tourney_name_Bogota
tourney_name_Bologna
tourney_name_Bolzano
tourney_name_Bordeaux
tourney_name_Boston
tourney_

In [10]:
# # Assume df is your DataFrame
# numeric_cols = data.columns
# n_cols = len(numeric_cols)

# # Set grid size
# n_rows = math.ceil(n_cols / 3)
# fig, axes = plt.subplots(n_rows, 3, figsize=(15, 5 * n_rows))
# axes = axes.flatten()

# # Plot each histogram
# for i, col in enumerate(numeric_cols):
#     sns.histplot(data=data, x=col, ax=axes[i], kde=False)
#     axes[i].set_title(f'{col}')

# # Hide unused subplots
# for j in range(i + 1, len(axes)):
#     fig.delaxes(axes[j])

# plt.tight_layout()
# plt.show()